### 오차역전파법 
: 가중치 매개변수의 기울기를 효율적으로 계산하는 것 

- 수식을 통한 것 / 계산 그래프를 통한 것 

#### 계산 그래프 
: 계산 과정을 그래프로 나타낸 것        


► 역전파 통해 '미분' 효율적 계산 


- 순전파 : 왼쪽에서 오른쪽으로 진행하는 계산


- 역전파 : 오른쪽에서 왼쪽으로 진행하는 계산


- 국소적 계산 : 전체가 아무리 복잡해도 각  노드에서는 단순한 계산에 집중하여 문제 단순화

#### 합성 함수 : 여러 함수로 구성된 함수

→ 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수  있다.

In [2]:
# 곱셉 계층 구현

class MulLayer:  # 곱셈 계층 
    def __init__(self):  # 인스턴스 변수 x, y 초기화
        self.x = None
        self.y = None

    def forward(self, x, y): # 순전파 / x, y 곱해서 반환
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout): # 역전파 / 미분(dout)에 순전파 시의 입력 값 유지하기 위해 사용
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy


apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)        

220.00000000000003


In [3]:
# 역전파 
# 호출 순서 forword()와 반대 / backword()가 받는 인수는 '순전파의 출력에 대한 미분'

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)


print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [ ]:
# 덧셈 계층

class AddLayer:
    def __init__(self): # 초기화가 필요없어서 아무것도 안함
        pass # 아무것도 하지말라는 명령어

    def forward(self, x, y): 
        out = x + y

        return out

    def backward(self, dout): # 상류에서 내려온 미분(dout)을 그대로 하류로 흘릴뿐
        dx = dout * 1
        dy = dout * 1

        return dx, dy


In [ ]:
from layer_naive import *

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


In [4]:
# 활성화 함수 계층 구현하기 
# ReLU 계층

class Relu:
    def __init__(self):
        self.mask = None  # mask라는 인스턴스 변수 가짐 / True, False로 구성된 넘파이 배열

    def forword(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout 

        return dx   

In [6]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [8]:
# 순전파 때의 입력 값이 0 이하면 역전파의 값은 0이 되어야한다.
# 역전파 때는 순전파 때 만들어둔 mask 써서 mask원소가 True인 곳에 상류에서 전파된 dout을 0으로 설정

mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


ReLU 계층은 전기 회로의 '스위치'

- 순전파 때 전류가 흐르고 있으면 스위치 ON, 흐르지 않으면 OFF
- 역전파 때 스위치가 ON이면 전류가 그대로 흐르고, OFF면 더 이상 흐르지 않는다.